In [2]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3 import PPO

from gym.wrappers import GrayScaleObservation

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
import numpy as np
import os
from stable_baselines3.common.callbacks import BaseCallback

import optuna

from stable_baselines3.common.evaluation import evaluate_policy

import os


env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)



log_dir = './log_dir2/'
os.makedirs(log_dir, exist_ok=True)

env = Monitor(env, log_dir)

env = GrayScaleObservation(env,keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4,channels_order='last')

In [2]:
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [3]:
def optimize_agent(trial):
    
    try:
        env = gym_super_mario_bros.make('SuperMarioBros-v0')
        env = JoypadSpace(env, SIMPLE_MOVEMENT)

        log_dir = './log_dir2/'
        os.makedirs(log_dir, exist_ok=True)

        env = Monitor(env, log_dir)

        env = GrayScaleObservation(env,keep_dim=True)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env,4,channels_order='last')

    
        model_params = optimize_ppo(trial) 
    

        tensorboard_log = r'./logs/'
        model = PPO("CnnPolicy", env, verbose=0,tensorboard_log=tensorboard_log,**model_params)
        # model.learn(total_timesteps=1000)
        model.learn(total_timesteps=200000)
    
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=5)
    
    
        env.close()
    
        #OPT_DIR  = r'F:\\300_RL_DEMO\\220_Super-Mario-2\\'
        OPT_DIR  = r'C:/Users/周靖雅/Desktop/RL/代码/RL_Mario3'
        
        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)
    
        return mean_reward    

    except Exception as e:
        return -1000
    

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=120)

D:\Programs\anaconda\envs\cudapy37\lib\site-packages\stable_baselines3\ppo\ppo.py:147: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3772`, after every 58 untruncated mini-batches, there will be a truncated mini-batch of size 60
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3772 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
D:\Programs\anaconda\envs\cudapy37\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
[I 2024-07-23 12:48:42,851] Finished trial#0 resulted in value: 741.0. Current best value is 741.0 with parameters: {'n_steps': 3772, 'gamma': 0.9314454580613634, 'learning_rate': 1.671615566410799e-05, 'clip_range': 0.14023642239809342, 'gae_lambda': 0.9549062192910633}.
D:\Programs\anaconda\envs\cudapy37\lib\site-packages\stable_ba

[I 2024-07-23 23:56:06,000] Finished trial#11 resulted in value: 741.0. Current best value is 1925.0 with parameters: {'n_steps': 3689, 'gamma': 0.8469821635688579, 'learning_rate': 5.443989010860513e-05, 'clip_range': 0.12925581231825378, 'gae_lambda': 0.8052062222826935}.
D:\Programs\anaconda\envs\cudapy37\lib\site-packages\stable_baselines3\ppo\ppo.py:147: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3601`, after every 56 untruncated mini-batches, there will be a truncated mini-batch of size 17
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3601 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2024-07-24 01:46:58,749] Finished trial#12 resulted in value: 1564.0. Current best value is 1925.0 with parameters: {'n_steps': 3689, 'gamma': 0.8469821635688579, 'learning_rate': 5.443989010860513e-05, 'clip_range': 0.12925581231825378, 'gae_lambda': 

[I 2024-07-24 18:01:50,706] Finished trial#22 resulted in value: 2180.0. Current best value is 2180.0 with parameters: {'n_steps': 4208, 'gamma': 0.8242870458415424, 'learning_rate': 9.923023306524416e-05, 'clip_range': 0.21582360473251122, 'gae_lambda': 0.8014658636642029}.
D:\Programs\anaconda\envs\cudapy37\lib\site-packages\stable_baselines3\ppo\ppo.py:147: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3957`, after every 61 untruncated mini-batches, there will be a truncated mini-batch of size 53
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3957 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2024-07-24 19:34:38,562] Finished trial#23 resulted in value: 2055.0. Current best value is 2180.0 with parameters: {'n_steps': 4208, 'gamma': 0.8242870458415424, 'learning_rate': 9.923023306524416e-05, 'clip_range': 0.21582360473251122, 'gae_lambda':

[I 2024-07-25 03:37:30,401] Finished trial#33 resulted in value: -1000.0. Current best value is 3149.0 with parameters: {'n_steps': 2451, 'gamma': 0.8856232603126483, 'learning_rate': 7.126626836931048e-05, 'clip_range': 0.25459705077650785, 'gae_lambda': 0.8153375352199685}.
D:\Programs\anaconda\envs\cudapy37\lib\site-packages\stable_baselines3\ppo\ppo.py:147: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2628`, after every 41 untruncated mini-batches, there will be a truncated mini-batch of size 4
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2628 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2024-07-25 05:14:39,418] Finished trial#34 resulted in value: 1098.0. Current best value is 3149.0 with parameters: {'n_steps': 2451, 'gamma': 0.8856232603126483, 'learning_rate': 7.126626836931048e-05, 'clip_range': 0.25459705077650785, 'gae_lambda':

[I 2024-07-25 15:38:06,443] Finished trial#44 resulted in value: 615.0. Current best value is 3149.0 with parameters: {'n_steps': 2451, 'gamma': 0.8856232603126483, 'learning_rate': 7.126626836931048e-05, 'clip_range': 0.25459705077650785, 'gae_lambda': 0.8153375352199685}.
D:\Programs\anaconda\envs\cudapy37\lib\site-packages\stable_baselines3\ppo\ppo.py:147: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3547`, after every 55 untruncated mini-batches, there will be a truncated mini-batch of size 27
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3547 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2024-07-25 15:38:09,331] Finished trial#45 resulted in value: -1000.0. Current best value is 3149.0 with parameters: {'n_steps': 2451, 'gamma': 0.8856232603126483, 'learning_rate': 7.126626836931048e-05, 'clip_range': 0.25459705077650785, 'gae_lambda':

D:\Programs\anaconda\envs\cudapy37\lib\site-packages\stable_baselines3\ppo\ppo.py:147: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3404`, after every 53 untruncated mini-batches, there will be a truncated mini-batch of size 12
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3404 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2024-07-25 20:27:55,435] Finished trial#57 resulted in value: -1000.0. Current best value is 3149.0 with parameters: {'n_steps': 2451, 'gamma': 0.8856232603126483, 'learning_rate': 7.126626836931048e-05, 'clip_range': 0.25459705077650785, 'gae_lambda': 0.8153375352199685}.
D:\Programs\anaconda\envs\cudapy37\lib\site-packages\stable_baselines3\ppo\ppo.py:147: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2306`, after every 36 untruncated mini-batches

In [3]:
study.best_params

NameError: name 'study' is not defined